In [1]:
import os, numpy as np
import pandas as pd

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras 
from sklearn import datasets

from sklearn.model_selection import train_test_split

from keras.utils import np_utils



/home/antoine/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Build a Keras Model

In [2]:

def create_model():
    from keras.models import Sequential
    from keras.layers.core import Dense, Activation, Dropout
    from keras.utils import np_utils

    model = Sequential()
    model.add(Dense(5, input_shape=(4,) , activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model



In [3]:
iris  = datasets.load_iris()
train_X, test_X, train_y, test_y = train_test_split(iris.data, iris.target, train_size=0.8, test_size=0.2, random_state=1960)


In [4]:

from keras.wrappers.scikit_learn import KerasClassifier

clf = KerasClassifier(build_fn=create_model, epochs=12, verbose=0)

print(train_X.shape , train_y.shape)
clf.fit(train_X, train_y, verbose=0)

(120, 4) (120,)


In [5]:
print(test_X.shape)
preds = clf.predict(test_X[0,:].reshape(1,4))
print(preds)


(30, 4)
[0]


# Generate SQL Code from the Model

In [6]:
import json, requests, base64, dill as pickle, sys



sys.setrecursionlimit(200000)
pickle.settings['recurse'] = False

# no luck for the web service... pickling feature of tensorflow and/or keras objects seems not to be a priority.
# there is a lot of github issues in the two projects when I search for pickle keyword!!!.

def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



def test_sql_gen(keras_regressor , metadata):
    import sklearn2sql.PyCodeGenerator as codegen
    cg1 = codegen.cAbstractCodeGenerator();
    lSQL = cg1.generateCodeWithMetadata(clf, metadata, dsn = None, dialect = "postgresql");
    return lSQL[0]


In [7]:
# commented .. see above
# pickle_data = pickle.dumps(clf)
# lSQL = test_ws_sql_gen(pickle_data)
# print(lSQL[0:2000])

In [8]:
lMetaData = {}
lMetaData['features'] = list(iris.feature_names);
#column name adaptation
lMetaData['features'] = [col.replace(" (cm)" , "_cm") for col in lMetaData['features'] ]

lMetaData["targets"] = ['TGT']
lMetaData['primary_key'] = 'KEY'
lMetaData['table'] = 'iris'

    
lSQL = test_sql_gen(clf , lMetaData)


/usr/local/lib/python3.6/dist-packages/xgboost
cGenerationWrapperFactory::createWrapper() <class 'keras.wrappers.scikit_learn.KerasClassifier'>
cClassifierMixin_CodeGenWrapper::setObject <class 'keras.wrappers.scikit_learn.KerasClassifier'>
BACKEND_DIALECT postgresql
CREATING_DATABASE_BACKEND_DSN_DIALECT 1.2.2 None postgresql
KERAS_GENERATE_EXPRESSION_START <class 'sqlalchemy.sql.selectable.Alias'> <class 'sklearn2sql.GenericModels.Keras.cSerializedKeras'>
{'mKerasData': <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f0963b7eda0>, 'mFeatureNames': ['sepal length_cm', 'sepal width_cm', 'petal length_cm', 'petal width_cm'], 'mClasses': array([0, 1, 2]), 'mMode': 'Classification'}
{'build_fn': <function create_model at 0x7f09b1e30ae8>, 'sk_params': {'epochs': 12, 'verbose': 0}, 'classes_': array([0, 1, 2]), 'n_classes_': 3, 'model': <keras.models.Sequential object at 0x7f0963b7edd8>}
ABSTRACT_GENERATE_CTE keras_input keras_input 5
GENERATING_LAYER 0 dense_1 <class 'keras.layers

In [9]:
print(lSQL)

WITH keras_input AS 
(SELECT "ADS"."KEY" AS "KEY", "ADS"."sepal length_cm" AS "sepal length_cm", "ADS"."sepal width_cm" AS "sepal width_cm", "ADS"."petal length_cm" AS "petal length_cm", "ADS"."petal width_cm" AS "petal width_cm" 
FROM iris AS "ADS"), 
layer_dense_1 AS 
(SELECT keras_input."KEY" AS "KEY", 0.0 + 0.024334847927093506 * keras_input."sepal length_cm" + -0.2723749279975891 * keras_input."sepal width_cm" + -0.680120587348938 * keras_input."petal length_cm" + 0.23243749141693115 * keras_input."petal width_cm" AS output_1, 0.0 + 0.11708617210388184 * keras_input."sepal length_cm" + -0.6165568828582764 * keras_input."sepal width_cm" + -0.5510092377662659 * keras_input."petal length_cm" + -0.2583366632461548 * keras_input."petal width_cm" AS output_2, -0.03478842228651047 + -0.37598344683647156 * keras_input."sepal length_cm" + 0.21327409148216248 * keras_input."sepal width_cm" + 0.2596365809440613 * keras_input."petal length_cm" + 0.15154574811458588 * keras_input."petal width_

# Execute the SQL Code

In [10]:
# save the dataset in a database table


import sqlalchemy as sa

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(iris.data);
lTable.columns = lMetaData['features']
lTable['TGT'] = iris.target
lTable['KEY'] = range(iris.data.shape[0])
lTable.to_sql(lMetaData['table'] , conn,   if_exists='replace', index=False)

In [11]:
sql_output = pd.read_sql(lSQL , conn);

In [12]:
sql_output.sample(12, random_state=1960)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,None,None,None,0.337471,0.334984,0.327545,-1.086276,-1.093672,-1.116129,0,0.337471
74,74,None,None,None,0.323255,0.343214,0.333530,-1.129312,-1.069401,-1.098021,1,0.343214
9,9,None,None,None,0.399572,0.164118,0.436309,-0.917360,-1.807169,-0.829404,2,0.436309
88,88,None,None,None,0.323255,0.343214,0.333530,-1.129312,-1.069401,-1.098021,1,0.343214
25,25,None,None,None,0.395616,0.174181,0.430203,-0.927311,-1.747658,-0.843499,2,0.430203
5,5,None,None,None,0.415942,0.120259,0.463799,-0.877209,-2.118105,-0.768305,2,0.463799
48,48,None,None,None,0.414225,0.125088,0.460687,-0.881346,-2.078736,-0.775037,2,0.460687
117,117,None,None,None,0.323255,0.343214,0.333530,-1.129312,-1.069401,-1.098021,1,0.343214
83,83,None,None,None,0.323255,0.343214,0.333530,-1.129312,-1.069401,-1.098021,1,0.343214
105,105,None,None,None,0.323255,0.343214,0.333530,-1.129312,-1.069401,-1.098021,1,0.343214


# Keras Prediction

In [13]:
keras_output = pd.DataFrame()
keras_output_key = pd.DataFrame(list(range(iris.data.shape[0])), columns=['KEY']);
keras_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
keras_output_proba = pd.DataFrame(clf.predict_proba(iris.data), columns=['Proba_0', 'Proba_1', 'Proba_2'])
keras_output = pd.concat([keras_output_key, keras_output_score, keras_output_proba] , axis=1)
for class_label in [0, 1, 2]:
    keras_output['LogProba_' + str(class_label)] = np.log(keras_output_proba['Proba_' + str(class_label)])
keras_output['Decision'] = clf.predict(iris.data)
keras_output.sample(12, random_state=1960)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.337471,0.334984,0.327545,-1.086276,-1.093672,-1.116129,0
74,74,NaN,NaN,NaN,0.323255,0.343214,0.333530,-1.129312,-1.069401,-1.098022,1
9,9,NaN,NaN,NaN,0.399572,0.164118,0.436309,-0.917360,-1.807169,-0.829404,2
88,88,NaN,NaN,NaN,0.323255,0.343214,0.333530,-1.129312,-1.069401,-1.098022,1
25,25,NaN,NaN,NaN,0.395616,0.174181,0.430203,-0.927311,-1.747658,-0.843499,2
5,5,NaN,NaN,NaN,0.415942,0.120259,0.463799,-0.877209,-2.118105,-0.768305,2
48,48,NaN,NaN,NaN,0.414225,0.125088,0.460687,-0.881346,-2.078736,-0.775037,2
117,117,NaN,NaN,NaN,0.323255,0.343214,0.333530,-1.129312,-1.069401,-1.098022,1
83,83,NaN,NaN,NaN,0.323255,0.343214,0.333530,-1.129312,-1.069401,-1.098022,1
105,105,NaN,NaN,NaN,0.323255,0.343214,0.333530,-1.129312,-1.069401,-1.098022,1


# Comparing the SQL and Keras Predictions

In [14]:
sql_keras_join = keras_output.join(sql_output , how='left', on='KEY', lsuffix='_keras', rsuffix='_sql')


In [15]:
sql_keras_join.head(12)

,KEY_keras,Score_0_keras,Score_1_keras,Score_2_keras,Proba_0_keras,Proba_1_keras,Proba_2_keras,LogProba_0_keras,LogProba_1_keras,LogProba_2_keras,...,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql,DecisionProba
0,0,NaN,NaN,NaN,0.412012,0.131214,0.456774,-0.886703,-2.030924,-0.783567,...,None,None,0.412012,0.131214,0.456774,-0.886703,-2.030924,-0.783567,2,0.456774
1,1,NaN,NaN,NaN,0.400532,0.161654,0.437814,-0.914963,-1.822296,-0.825961,...,None,None,0.400532,0.161654,0.437814,-0.914963,-1.822296,-0.825961,2,0.437814
2,2,NaN,NaN,NaN,0.407262,0.144047,0.448691,-0.898298,-1.937615,-0.801421,...,None,None,0.407262,0.144047,0.448691,-0.898298,-1.937615,-0.801421,2,0.448691
3,3,NaN,NaN,NaN,0.400165,0.162598,0.437237,-0.915879,-1.816474,-0.827279,...,None,None,0.400165,0.162598,0.437237,-0.915879,-1.816474,-0.827279,2,0.437237
4,4,NaN,NaN,NaN,0.413835,0.126176,0.459989,-0.882289,-2.070075,-0.776553,...,None,None,0.413835,0.126176,0.459989,-0.882289,-2.070074,-0.776553,2,0.459989
5,5,NaN,NaN,NaN,0.415942,0.120259,0.463799,-0.877209,-2.118105,-0.768305,...,None,None,0.415942,0.120259,0.463799,-0.877209,-2.118105,-0.768305,2,0.463799
6,6,NaN,NaN,NaN,0.410219,0.136104,0.453677,-0.891064,-1.994335,-0.790371,...,None,None,0.410219,0.136104,0.453677,-0.891064,-1.994335,-0.790371,2,0.453677
7,7,NaN,NaN,NaN,0.407786,0.142649,0.449564,-0.897012,-1.947366,-0.799476,...,None,None,0.407786,0.142649,0.449564,-0.897012,-1.947366,-0.799476,2,0.449564
8,8,NaN,NaN,NaN,0.397188,0.170202,0.432610,-0.923346,-1.770771,-0.837918,...,None,None,0.397188,0.170202,0.432610,-0.923346,-1.770771,-0.837918,2,0.432610
9,9,NaN,NaN,NaN,0.399572,0.164118,0.436309,-0.917360,-1.807169,-0.829404,...,None,None,0.399572,0.164118,0.436309,-0.917360,-1.807169,-0.829404,2,0.436309


In [16]:
condition = (sql_keras_join.Decision_sql != sql_keras_join.Decision_keras)
sql_keras_join[condition]

,KEY_keras,Score_0_keras,Score_1_keras,Score_2_keras,Proba_0_keras,Proba_1_keras,Proba_2_keras,LogProba_0_keras,LogProba_1_keras,LogProba_2_keras,...,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql,DecisionProba
